<a href="https://colab.research.google.com/github/GusevMihail/seamless_textute_generator/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U albumentations

Requirement already up-to-date: albumentations in /usr/local/lib/python3.7/dist-packages (0.5.2)


In [2]:
import os.path
import sys
if 'google' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir data
    !mkdir data/train
    if os.path.exists('data/train'):
        !cp /content/drive/MyDrive/Colab/seamless_textute_generator/data/concrete_maps_1K.zip data
        !unzip -q -n data/concrete_maps_1K.zip -d data/train

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/train’: File exists


In [3]:
from typing import Tuple, Any
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import torch
import torch.nn as nn
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import cv2

import albumentations as albu

# Dataset

In [4]:
data_folder = r'data/train'
image_names = tuple(f for f in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, f)))

In [5]:
from torch.utils import data

class TextureDataset(data.Dataset):

    def __init__(self, root_path: str, file_list: list, transforms: Any=None, 
                 cross_koeff: float=0.1, im_size: Tuple[int, int]=(224, 224)):
        super().__init__()
        self.root_path = root_path
        self.file_list = file_list
        self.transforms = transforms
        self.im_size = im_size
        self.cross_coeff = cross_koeff

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index: int) -> Tuple[np.array, np.array]:
        image_path = os.path.join(self.root_path, self.file_list[index])

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        

        if self.transforms is not None:
            transformed = self.transforms(image=image)
            image = transformed['image']
            # mask = transformed['mask']

        # X, mask = self.cut_cross(image)
        # return X, mask, image

        return image, image, image

        
    
    def cut_cross(self, img: np.array)->Tuple[np.array, np.array]:
        height, width, _ = img.shape
        # Размеры креста
        cross_height = round(height * self.cross_coeff / 2)
        cross_width = round(width * self.cross_coeff / 2)

        # Индексы вырезания креста
        start_height_idx = round(height / 2 - cross_height)
        end_height_idx = round(height / 2 + cross_height)

        start_width_idx = round(width / 2 - cross_width)
        end_width_idx = round(width / 2 + cross_width)

        # Вырежем крест
        X = img.copy()
        X[start_height_idx : end_height_idx] = 0
        X[:, start_width_idx : end_width_idx] = 0
        # Определим маску креста
        mask = np.zeros(X.shape[:2]).astype(float)
        mask[start_height_idx : end_height_idx] = 1.
        mask[:, start_width_idx : end_width_idx] = 1.
        return X, mask

In [8]:
from albumentations.pytorch import ToTensorV2

In [9]:
resize_transform = albu.Compose([albu.Resize(224, 224),
                                 albu.Normalize(),
                                 ToTensorV2()])

In [10]:
dataset = TextureDataset(data_folder, image_names, resize_transform)

In [ ]:
# look at the image
np.random.seed(42)

fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(8, 8),
                       sharey=True, sharex=True)

for fig_x in ax.flatten():
    i = np.random.choice(len(dataset), 1)[0]
    im , _, _ = dataset[i]

    fig_x.imshow(im)
    # if img_label is not None:
    #     fig_x.set_title(img_label)
    fig_x.grid(False)

# Model

In [11]:
class FullyConv(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self._model = nn.Sequential(
             nn.Conv2d(in_channels, 16, 3, 2, self.same_padding(3)),   # 224
             nn.ReLU(),
             nn.BatchNorm2d(16),
             nn.Conv2d(16, 32, 3, 2, self.same_padding(3)),            # 112
             nn.ReLU(),
             nn.BatchNorm2d(32),
             nn.Conv2d(32, 64, 3, 2, self.same_padding(3)),            # 56
             nn.ReLU(),
             nn.BatchNorm2d(64),
             nn.Conv2d(64, 128, 3, 2, self.same_padding(3)),           # 28
             nn.ReLU(),
             nn.BatchNorm2d(128),
             nn.Conv2d(128, 256, 3, 2, self.same_padding(3)),          # 14
             nn.ReLU(),
             nn.AdaptiveAvgPool2d(1),                                  # 7
             nn.Conv2d(256, 512, 1, 1, self.same_padding(1)),          # 1
             nn.ReLU(),
             nn.Dropout(0.2),
             nn.Conv2d(512, out_channels, 1, 1, self.same_padding(1))) # 1 
        
    @staticmethod
    def same_padding(kernel_size):
        return (kernel_size - 1) // 2
    
    def forward(self, X):
        return self._model(X)

In [12]:
model = FullyConv(3, 224*3)

In [13]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset, batch_size=1)

In [14]:
for i in train_loader:
    print(i[0].size())
    print(model(i[0]).size())
    break


torch.Size([1, 3, 224, 224])
torch.Size([1, 672, 1, 1])


In [17]:
tuple(model(i[0]).size())

(1, 672, 1, 1)

In [18]:
from copy import deepcopy

# import torch
# import torch.nn as nn
from tqdm import tqdm

# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def do_epoch(model, optimizer, loss_func, data_loader,
             mode='T', metric=None, title=None):
    """
    Compute one epoch
    :param model: (nn.Module) model
    :param optimizer: (torch.optim) optimization method. Ignored if mode='V'
    :param loss_func: (func) loss functions
    :param data_loader: (MyDataLoader) val batches generator (X, y). Default None
    :param mode: (str) 'T' - Train or 'V' - Validate. Default 'T'
    :param metric: (func) target metric
    :param title: (str) description in progress bar
    :return:
        epoch_loss: mean loss
        epoch_metric: mean metric
    """
    if mode not in ['V', 'T']:
        raise ValueError('mode should be "T" or "V"')
    # History
    epoch_loss = 0.
    epoch_metric = 0.


    with tqdm(total=len(data_loader)) as progress_bar:
        for ind, (X, mask, y) in enumerate(data_loader, 1):
            description = ''
            if title is not None:
                description += title
            description += f'Mode: {mode} |'
            # X_tens, y_tens = torch.as_tensor(X, dtype=torch.float, device=DEVICE), \
            #                  torch.as_tensor(y, dtype=torch.long, device=DEVICE)
            X_tens = X
            y_tens = y
            y_shape = tuple(y.size())
            predict = model(X_tens).view(*y_shape)
            loss = loss_func(predict, y_tens)
            epoch_loss += loss.item()
            description += f'Loss: {epoch_loss / ind: 7.4} |'

            # backward
            if mode == 'T':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            #  metric calculate
            if metric is not None:
                epoch_metric += metric(predict, y_tens)
                description += f'Metric: {epoch_metric / ind: 7.4} |'

            progress_bar.set_description(description)
            progress_bar.update()
    return epoch_loss / len(data_loader), epoch_metric / len(data_loader)


def train(model, train_loader, loss_func, optimizer, epoch_count=10,
          metric=None, val_loader=None, scheduler=None):
    """
    Training model
    :param model: (torch.nn.Module) model for train
    :param train_loader: (MyDataLoader) train batches generator (X, y)
    :param loss_func: (func) loss functions
    :param optimizer: (torch.optim) optimization method
    :param epoch_count: (int) epochs count. Default 10
    :param metric: (func) target metric
    :param val_loader: (MyDataLoader) val batches generator (X, y). Default None
    :param scheduler: (torch.utils)
    :return:
        history_info: dict of training history consist "Tloss", "Tmetric",
                    "Vloss", "Vmetric"
        best_model_param: model parameters at the highest Vmetric value
    """
    # Train_history
    history_info = {'Tloss': [], 'Tmetric': [],
                    'Vloss': [], 'Vmetric': []}
    # best Val_score and model params
    best_score = 0.
    best_model_param = {}

    datasets = {}

    if train_loader is not None:
        datasets.update({'T': train_loader})
    if val_loader is not None:
        datasets.update({'V': val_loader})

    for epoch in range(epoch_count):
        for mode, data in datasets.items():
            # title for progress bar
            title = f'[{epoch+1: 3}/{epoch_count}]|'
            model.train(mode == 'T')
            epoch_loss, epoch_metric = \
                do_epoch(model, optimizer, loss_func, data,
                         mode, metric, title)
            history_info[mode + 'loss'].append(epoch_loss)
            history_info[mode + 'metric'].append(epoch_metric)

            if metric is not None:
                # save best metric value and model parameters
                if best_score < epoch_metric and mode == 'V':
                    best_score = epoch_metric
                    best_model_param = deepcopy(model.state_dict())

            # scheduler step
            if scheduler is not None and mode == 'V':
                scheduler.step(epoch_metric)

    return history_info, best_model_param


def accuracy(predict_proba, ground_truth):
    """
    Compute accuracy
    :param predict_proba: (torch.Tensor) prediction probability 2D dim
    :param ground_truth: (torch.Tensor) ground truth labels
    :return: (float) accuracy
    """
    label_index = torch.argmax(predict_proba, dim=-1)
    true_predict = (label_index == ground_truth).sum().item()
    return true_predict / ground_truth.size()[0]